# 📈 ML4T Intern Guide: Crypto ML Model using CoinGecko
Apply the workflow from *Machine Learning for Algorithmic Trading* by Stefan Jansen using CoinGecko data.

## Step 1: Get Crypto Data from CoinGecko

In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
import time
from datetime import datetime, timedelta

cg = CoinGeckoAPI()

def get_price_history(coin_id, days=90):
    end = int(time.time())
    start = end - days * 86400
    data = cg.get_coin_market_chart_range_by_id(id=coin_id, vs_currency='usd',
                                                from_timestamp=start, to_timestamp=end)
    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    prices['timestamp'] = pd.to_datetime(prices['timestamp'], unit='ms')
    return prices.set_index('timestamp').resample('1D').mean()

btc_prices = get_price_history('bitcoin')
btc_prices['return'] = btc_prices['price'].pct_change()
btc_prices.tail()

## Step 2: Feature Engineering

In [ ]:
btc_prices['momentum_7d'] = btc_prices['price'].pct_change(7)
btc_prices['volatility_7d'] = btc_prices['return'].rolling(7).std()
btc_prices['z_score'] = (btc_prices['price'] - btc_prices['price'].rolling(20).mean()) / btc_prices['price'].rolling(20).std()
btc_prices.dropna(inplace=True)
btc_prices.tail()

## Step 3: Train ML Model

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

features = ['momentum_7d', 'volatility_7d', 'z_score']
X = btc_prices[features]
y = btc_prices['return'].shift(-1)

X.dropna(inplace=True)
y = y.loc[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)

model = CatBoostRegressor(verbose=0)
model.fit(X_train, y_train)

## Step 4: Predict & Evaluate

In [ ]:
btc_prices.loc[X_test.index, 'prediction'] = model.predict(X_test)
btc_prices[['return', 'prediction']].plot(figsize=(12, 6), title='Predicted vs Actual Returns');